In [1]:
import pandas as pd
import numpy as np

# Settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

pd.options.display.float_format = '{:,.2f}'.format

In [2]:
# Import mappning tables
exposures = pd.read_excel('kala_eba_tr.xlsx', sheet_name='Exposures')
portfolio = pd.read_excel('kala_eba_tr.xlsx', sheet_name='Portfolio')
country = pd.read_excel('kala_eba_tr.xlsx', sheet_name='Country')
status = pd.read_excel('kala_eba_tr.xlsx', sheet_name='Status')
perf_status = pd.read_excel('kala_eba_tr.xlsx', sheet_name='Perf_Status')
institutions = pd.read_excel('kala_eba_tr.xlsx', sheet_name='List of institutions')

# import core dataset
df = pd.read_csv('tr_cre.csv', low_memory=False)

In [3]:
# Create final dataset
result = pd.merge(df, exposures, on='Exposure', how='left')
result = pd.merge(result, portfolio, on='Portfolio', how='left')
result = pd.merge(result, country, on='Country', how='left')
result = pd.merge(result, status, on='Status', how='left')
result = pd.merge(result, perf_status, on='Perf_Status', how='left')
result = pd.merge(result, institutions, on='LEI_Code', how='left')

result.head()

,LEI_Code,NSA,Period,Item,Label,Portfolio,Country,Country_rank,Exposure,Status,Perf_Status,NACE_codes,Amount,Footnote,Row,Column,Sheet,exposure_label,portfolio_label,country_label,ISO_code,status_label,perf_status_label,institution_country,Desc_country,Name,Finrep,Fin_year_end
0,0W2PZJM8XOY22M4GG883,DE,202209,2320502,Original Exposure - by exposure class (SA_and_IRB),2,0,0,103,0,0,0,419.98,NaN,10,4,Credit Risk_IRB_a,Central governments or central banks,IRB,Total / No breakdown,00,No breakdown by status,No breakdown by Perf_status,DE,Germany,DekaBank Deutsche Girozentrale,Yes - IFRS,31/12
1,0W2PZJM8XOY22M4GG883,DE,202209,2320502,Original Exposure - by exposure class (SA_and_IRB),2,0,0,203,0,0,0,"16,941.37",NaN,11,4,Credit Risk_IRB_a,Institutions,IRB,Total / No breakdown,00,No breakdown by status,No breakdown by Perf_status,DE,Germany,DekaBank Deutsche Girozentrale,Yes - IFRS,31/12
2,0W2PZJM8XOY22M4GG883,DE,202209,2320502,Original Exposure - by exposure class (SA_and_IRB),2,0,0,303,0,0,0,"32,797.84",NaN,12,4,Credit Risk_IRB_a,Corporates,IRB,Total / No breakdown,00,No breakdown by status,No breakdown by Perf_status,DE,Germany,DekaBank Deutsche Girozentrale,Yes - IFRS,31/12
3,0W2PZJM8XOY22M4GG883,DE,202209,2320502,Original Exposure - by exposure class (SA_and_IRB),2,0,0,404,0,0,0,0.00,NaN,15,4,Credit Risk_IRB_a,Retail,IRB,Total / No breakdown,00,No breakdown by status,No breakdown by Perf_status,DE,Germany,DekaBank Deutsche Girozentrale,Yes - IFRS,31/12
4,0W2PZJM8XOY22M4GG883,DE,202209,2320502,Original Exposure - by exposure class (SA_and_IRB),2,0,0,606,0,0,0,400.26,NaN,23,4,Credit Risk_IRB_a,Equity exposures,IRB,Total / No breakdown,00,No breakdown by status,No breakdown by Perf_status,DE,Germany,DekaBank Deutsche Girozentrale,Yes - IFRS,31/12


In [4]:
result['Label'].unique()

array(['Original Exposure - by exposure class (SA_and_IRB)',
       'Original Exposure - SME - by exposure class (SA_and_IRB)',
       'Original Exposure - Corporates - Specialised Lending (IRB)',
       'Original Exposure - Retail - by type (IRB)',
       'Original Exposure - Retail_non SME - by type (IRB)',
       'Original Exposure - of which_DEFAULTED - by exposure class (SA_and_IRB)',
       'Original Exposure - SME_of which_DEFAULTED - by exposure class (SA_and_IRB)',
       'Original Exposure - Corporates_Specialised Lending - of which_DEFAULTED (IRB)',
       'Original Exposure - Retail of which_DEFAULTED - by type (IRB)',
       'Original Exposure - Retail - non SME - of which_DEFAULTED - by type (IRB)',
       'Exposure value - by exposure class (SA_and_IRB)',
       'Exposure value - SME - by exposure class (SA_and_IRB)',
       'Exposure value - Corporates_Specialised Lending (IRB)',
       'Exposure value - Retail - by type (IRB)',
       'Exposure value - Retail - non SME

In [5]:
# Check the number of observations per row
idx_1 = result['Desc_country'] == 'Netherlands'
idx_2 = result['country_label'] == 'Netherlands'
idx_3 = result['Label'] == 'Exposure value - by exposure class (SA_and_IRB)'
idx_4 = result['Period'] == 202306

test = result[idx_1 & idx_2 & idx_3 & idx_4].groupby(['exposure_label', 'portfolio_label', 'Name']).agg({'Amount': ['sum', 'count']})
assert test[test.columns[-1]].max() == 1, 'Your filtering criteria is likely to be deficient'

In [6]:
# Create base table
idx_1 = result['Desc_country'] == 'Netherlands'
idx_2 = result['country_label'] == 'Netherlands'
idx_3 = result['Label'] == 'Exposure value - by exposure class (SA_and_IRB)'
idx_4 = result['Period'] == 202306
idx_5 = result['status_label'] == 'Non defaulted assets'
idx_6 = result['perf_status_label'] == 'Performing'

kala = pd.pivot_table(result[idx_1 & idx_2 & idx_3 & idx_4], 
                      values='Amount', 
                      index=['exposure_label', 'portfolio_label'], 
                      columns=['Name'], 
                      aggfunc="sum", 
                      margins=True,
                      fill_value=0).reset_index()

kala
kala.sort_values(by=kala.columns[-1], ascending=False)

Name,exposure_label,portfolio_label,ABN AMRO Bank N.V.,BNG Bank N.V.,Coöperatieve Rabobank U.A.,ING Groep N.V.,Nederlandse Waterschapsbank N.V.,de Volksbank N.V.,All
21,All,,"299,083.58","119,120.74","428,063.15","269,483.53","79,876.43","65,530.58","1,261,158.02"
18,Retail,IRB,"160,710.38",0.00,"220,833.77","125,288.79",0.00,"50,226.04","557,058.98"
1,Central governments or central banks,SA,"63,051.59","72,957.38",979.72,"57,644.55","59,933.91","11,751.62","266,318.77"
4,Corporates,IRB,"62,774.26",0.00,"98,643.54","78,991.62",0.00,0.00,"240,409.43"
0,Central governments or central banks,IRB,0.00,0.00,"96,187.36",0.00,0.00,0.00,"96,187.36"
17,Regional governments or local authorities,SA,17.77,"34,872.10",0.00,4.95,"15,973.84",101.29,"50,969.95"
5,Corporates,SA,"2,995.66","6,818.81","1,780.72","1,020.40","2,446.10",908.78,"15,970.46"
10,Institutions,IRB,"1,649.65",0.00,"4,804.63","4,536.79",0.00,0.00,"10,991.06"
19,Retail,SA,"2,645.46",0.00,"1,579.99",156.22,0.00,290.98,"4,672.66"
11,Institutions,SA,"3,322.16",163.76,55.55,54.17,388.30,628.09,"4,612.03"


In [7]:
idx_1 = result['Desc_country'] == 'Netherlands'
idx_2 = result['country_label'] == 'Netherlands'
idx_3 = result['Label'].isin(['Exposure value - by exposure class (SA_and_IRB)', 
                              'Risk exposure amount - by exposure class (SA_and_IRB)', 
                              'Risk Exposure amount - of which_DEFAULTED  - by exposure class (IRB)'])
idx_4 = result['Period'] == 202306
idx_5 = result['status_label'] == 'Non defaulted assets'
idx_6 = result['perf_status_label'] == 'Performing'

kala2 = pd.pivot_table(result[idx_1 & idx_2 & idx_3 & idx_4], 
                      values='Amount', 
                      index=['exposure_label', 'portfolio_label', 'Name', 'country_label'], 
                      columns=['Label'], 
                      aggfunc="sum", 
                      margins=False,
                      fill_value=0).reset_index()


# Create a rank based on the summed 'Exposure value', sorted by size
grouped = kala2.groupby('exposure_label')['Exposure value - by exposure class (SA_and_IRB)'].sum()
rank = grouped.sort_values(ascending=False).rank(method='dense', ascending=False).astype(int)

# Map the ranks back to the original DataFrame
kala2['exposure_type_size'] = kala2['exposure_label'].map(rank)

kala2 = kala2.rename(columns={
    'Exposure value - by exposure class (SA_and_IRB)': 'exposure_value',
    'Risk exposure amount - by exposure class (SA_and_IRB)': 'risk_exposure',
    'Risk Exposure amount - of which_DEFAULTED  - by exposure class (IRB)': 'defaulted_risk_exposure',
})

kala2['risk_weight'] = kala2['risk_exposure'] / kala2['exposure_value'] 


kala2.sort_values(by='exposure_type_size', ascending=True).reset_index().head(20)

Label,index,exposure_label,portfolio_label,Name,country_label,exposure_value,defaulted_risk_exposure,risk_exposure,exposure_type_size,risk_weight
0,107,Retail,SA,ING Groep N.V.,Netherlands,156.22,0.00,97.10,1,0.62
1,100,Retail,IRB,ABN AMRO Bank N.V.,Netherlands,"160,710.38","1,623.85","17,262.55",1,0.11
2,101,Retail,IRB,Coöperatieve Rabobank U.A.,Netherlands,"220,833.77","1,015.27","24,745.59",1,0.11
3,102,Retail,IRB,ING Groep N.V.,Netherlands,"125,288.79","1,864.89","17,158.71",1,0.14
4,103,Retail,IRB,de Volksbank N.V.,Netherlands,"50,226.04",618.12,"10,131.71",1,0.20
5,104,Retail,SA,ABN AMRO Bank N.V.,Netherlands,"2,645.46",0.00,"1,984.09",1,0.75
6,105,Retail,SA,BNG Bank N.V.,Netherlands,0.00,0.00,0.00,1,NaN
7,106,Retail,SA,Coöperatieve Rabobank U.A.,Netherlands,"1,579.99",0.00,"1,057.70",1,0.67
8,108,Retail,SA,Nederlandse Waterschapsbank N.V.,Netherlands,0.00,0.00,0.00,1,NaN
9,109,Retail,SA,de Volksbank N.V.,Netherlands,290.98,0.00,174.70,1,0.60


In [8]:
z = pd.pivot_table(kala2, 
                      values='risk_weight', 
                      index=['exposure_label', 'portfolio_label', 'country_label', 'exposure_type_size'], 
                      columns=['Name'], 
                      aggfunc="sum", 
                      margins=False,
                      fill_value=np.nan).reset_index()

z.sort_values(by='exposure_type_size', ascending=True).head(20)

Name,exposure_label,portfolio_label,country_label,exposure_type_size,ABN AMRO Bank N.V.,BNG Bank N.V.,Coöperatieve Rabobank U.A.,ING Groep N.V.,Nederlandse Waterschapsbank N.V.,de Volksbank N.V.
18,Retail,IRB,Netherlands,1,0.11,NaN,0.11,0.14,NaN,0.20
19,Retail,SA,Netherlands,1,0.75,0.00,0.67,0.62,0.00,0.60
0,Central governments or central banks,IRB,Netherlands,2,0.00,NaN,0.00,0.00,NaN,0.00
1,Central governments or central banks,SA,Netherlands,2,0.00,0.00,1.72,0.00,0.00,0.01
4,Corporates,IRB,Netherlands,3,0.54,NaN,0.41,0.29,NaN,0.00
5,Corporates,SA,Netherlands,3,0.75,0.86,0.93,0.98,0.90,0.86
17,Regional governments or local authorities,SA,Netherlands,4,0.00,0.00,0.00,0.20,0.00,0.00
11,Institutions,SA,Netherlands,5,0.23,0.21,0.90,0.26,0.35,0.29
10,Institutions,IRB,Netherlands,5,0.28,NaN,0.21,0.16,NaN,0.00
7,Equity exposures,IRB,Netherlands,6,3.00,NaN,2.57,2.37,NaN,0.00


In [9]:
z = pd.pivot_table(kala2, 
                      values='exposure_value', 
                      index=['exposure_label', 'portfolio_label', 'country_label', 'exposure_type_size'], 
                      columns=['Name'], 
                      aggfunc="sum", 
                      margins=False,
                      fill_value=np.nan).reset_index()

z.sort_values(by='exposure_type_size', ascending=True).reset_index(drop=True).head(20)

Name,exposure_label,portfolio_label,country_label,exposure_type_size,ABN AMRO Bank N.V.,BNG Bank N.V.,Coöperatieve Rabobank U.A.,ING Groep N.V.,Nederlandse Waterschapsbank N.V.,de Volksbank N.V.
0,Retail,IRB,Netherlands,1,"160,710.38",NaN,"220,833.77","125,288.79",NaN,"50,226.04"
1,Retail,SA,Netherlands,1,"2,645.46",0.00,"1,579.99",156.22,0.00,290.98
2,Central governments or central banks,IRB,Netherlands,2,0.00,NaN,"96,187.36",0.00,NaN,0.00
3,Central governments or central banks,SA,Netherlands,2,"63,051.59","72,957.38",979.72,"57,644.55","59,933.91","11,751.62"
4,Corporates,IRB,Netherlands,3,"62,774.26",NaN,"98,643.54","78,991.62",NaN,0.00
5,Corporates,SA,Netherlands,3,"2,995.66","6,818.81","1,780.72","1,020.40","2,446.10",908.78
6,Regional governments or local authorities,SA,Netherlands,4,17.77,"34,872.10",0.00,4.95,"15,973.84",101.29
7,Institutions,SA,Netherlands,5,"3,322.16",163.76,55.55,54.17,388.30,628.09
8,Institutions,IRB,Netherlands,5,"1,649.65",NaN,"4,804.63","4,536.79",NaN,0.00
9,Equity exposures,IRB,Netherlands,6,778.09,NaN,"2,461.22",411.51,NaN,0.00
